In [ ]:
import pandas as pd
import ast
import re

In [ ]:
df = pd.read_csv('../data/cleaned.csv', low_memory=False)
df['categories'] = df['categories'].apply(
    lambda c: ast.literal_eval(c)
)
df.head()

In [ ]:
# Group categories into broader ones
def group_war(categories):
    if isinstance(categories, str):
        listval = ast.literal_eval(categories)
    elif isinstance(categories, list):
        listval = categories
    listval = list(map(
        lambda x: re.sub('^.*War.*$', 'War', x),
        listval
    ))
    return listval

def group_value(values, old_value, new_value):
    if isinstance(values, str):
        listval = ast.literal_eval(values)
    elif isinstance(values, list):
        listval = values
    listval = list(map(
        lambda x: x.replace(old_value, new_value),
        listval
    ))
    return listval

df2 = df.copy()
df2['categories'] = df2['categories'].apply(
    group_war
)

category_mappings = [
    ('Historical Setting', 'American West'),
    ('Historical Setting', 'Ancient'),
    ('Historical Setting', 'Medieval'),
    ('Historical Setting', 'Napoleonic'),
    ('Historical Setting', 'Post-Napoleonic'),
    ('Historical Setting', 'Prehistoric'),
    ('Historical Setting', 'Renaissance'),
    ('Bluffing / Negotiation', 'Bluffing'),
    ('Bluffing / Negotiation', 'Negotiation'),
    ('Books', 'Book'),
    ('Books', 'Comic Book / Strip'),
    ('Books', 'Novel-based'),
    ('Economy / Industry', 'Economic'),
    ('Economy / Industry', 'Farming'),
    ('Economy / Industry', 'Industry / Manufacturing')
]

for mapping in category_mappings:
    df2['categories'] = df2['categories'].apply(
        lambda x: group_value(x, mapping[1], mapping[0])
    )

# Remove duplciates
df2['categories'] = df2['categories'].apply(
    lambda x: list(set(x))
)

def metric_by_year(data, metric):
    new_df = data[['year', metric]].copy()
    new_df = new_df.explode(metric).reset_index(drop=True)
    return new_df
    
cats_by_year = metric_by_year(df2, 'categories')
cats_by_year.head()


In [ ]:
# Group mechanics into broader ones
mechanics_mappings = [
    ('Auction', 'Auction: Once Around'),
    ('Auction', 'Closed Economy Auction'),
    ('Auction', 'Constrained Bidding'),
    ('Auction', 'Auction/Bidding'),
    ('Auction', 'Auction: Dutch Priority'),
    ('Auction', 'Auction: Dutch'),
    ('Auction', 'Auction: English'),
    ('Auction', 'Auction: Sealed Bid'),
    ('Auction', 'Auction: Turn Order Until Pass'),
    ('Auction', 'Auction: Multiple Lot'),
    ('Auction', 'Turn Order: Auction'),
    ('Auction', 'Auction Compensation'),
    ('Auction', 'Auction: Fixed Placement'),
    ('Auction', 'Auction: Dexterity'),
    ('Drafting', 'Action Drafting'),
    ('Drafting', 'Closed Drafting'),
    ('Drafting', 'Open Drafting'),
    ('Grid-Based', 'Hexagon Grid'),
    ('Grid-Based', 'Square Grid'),
    ('Grid-Based', 'Grid Movement')
]

for mapping in mechanics_mappings:
    df2['mechanics'] = df2['mechanics'].apply(
        lambda x: group_value(x, mapping[1], mapping[0])
    )

# Remove duplciates
df2['mechanics'] = df2['mechanics'].apply(
    lambda x: list(set(x))
)
mechs_by_year = metric_by_year(df2, 'mechanics')
df2.head()

In [ ]:
# Order metrics in descending order so we know which might be the interesting
# values to plot

def get_counts(data, metric):
    new_df = data[[metric]]
    new_df['val'] = 1
    new_df = new_df.groupby(metric).sum()
    return new_df

cat_counts = get_counts(cats_by_year, 'categories')
mech_counts = get_counts(mechs_by_year, 'mechanics')
display(cat_counts.sort_values('val', ascending=False).head(20))
display(mech_counts.sort_values('val', ascending=False).head(20))

In [ ]:
# Pivot the yearly data so it's plottable, and limit to years 1960-2023
def pivot(data, metric):
    new_df = data.copy()
    new_df['val'] = 1
    new_df = new_df.pivot_table(index='year', columns=metric, values='val', aggfunc='sum', fill_value=0)
    new_df = new_df.loc[new_df.index >= 1960].copy()
    new_df = new_df.loc[new_df.index < 2024].copy()
    return new_df

# Pivot and plot categories
cat_pivot = pivot(cats_by_year, 'categories')
cat_pivot[[
    'Card Game', 'War', "Children's Game", 'Party Game', 'Dice',
    'Abstract Strategy', 'Fantasy', 'Historical Setting', 'Educational',
    'Economy / Industry', 'Bluffing / Negotiation', 'Print & Play'
]].plot(title='Game Categories by Publication Year')

# Pivot and plot mechanics
mech_pivot = pivot(mechs_by_year, 'mechanics')
mech_pivot[[
    'Dice Rolling', 'Roll / Spin and Move', 'Hand Management', 'Set Collection',
    'Drafting', 'Simulation', 'Variable Player Powers', 'Cooperative Game',
    'Cooperative Game', 'Tile Placement', 'Grid-Based'
]].plot(title='Game Mechanics by Publication Year')